In [1]:
import os
import sys
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pandas_profiling import ProfileReport
from dateutil.parser import parse


project_dir = os.path.dirname(os.path.abspath('.')) # get the path of the project, if doesn't work use os.getcwd() instead and use rfind to find the project dir
sys.path.append(os.path.join(project_dir, 'src', 'data'))

from data_manipulatorr import get_input_data, clean_datetime_cols

bucket = 'eseit_bigdata'
directory = 'bigdata'

from google.cloud import storage

%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'data_manipulatorr'

In [3]:
def list_blobs(bucket_name, directory= "BIGDATA/DATA/processed/"):
    """lists all the blobs in the bucket,"""
    ##bucket_name = "especializacion_bigdata_wilfredo"
    
    storage_client = storage.Client()
    
    # Note: client_lists_blobs requires at least package version 
    blobs = storage_client.list_blobs(
        bucket_name,
        prefix = directory,
        delimiter = '/'
    )
    
    list_files = list()
    
    for blob in blobs:
        print(blob.name)
        if '.csv' in blob.name:
            list_files.append(blob.name)
        
    return list_files

In [ ]:
list_files = list_blobs(bucket_name = bucket)

BIGDATA/DATA/processed/
BIGDATA/DATA/processed/clean_llamadas_123_agosto_2019.csv
BIGDATA/DATA/processed/clean_llamadas_123_diciembre_2019.csv
BIGDATA/DATA/processed/clean_llamadas_123_julio_2019.csv
BIGDATA/DATA/processed/clean_llamadas_123_noviembre_2019.csv
BIGDATA/DATA/processed/clean_llamadas_123_octubre_2019.csv
BIGDATA/DATA/processed/clean_llamadas_123_septiembre_2019.csv


In [ ]:
list_data = list()

for file in list_files:
    print(file.split('/'))
    list_strings = file.split('/')
    
    data = get_input_data(
        bucket = bucket,
        initial_directory = directory,
        step = 'processed',
        filename = list_strings[-1]
    )
    
    list_data.append(data)
    
print('Total Dataframes: {}'.format(len(list_data)))

['BIGDATA', 'DATA', 'processed', 'clean_llamadas_123_agosto_2019.csv']
['BIGDATA', 'DATA', 'processed', 'clean_llamadas_123_diciembre_2019.csv']
['BIGDATA', 'DATA', 'processed', 'clean_llamadas_123_julio_2019.csv']
['BIGDATA', 'DATA', 'processed', 'clean_llamadas_123_noviembre_2019.csv']
['BIGDATA', 'DATA', 'processed', 'clean_llamadas_123_octubre_2019.csv']
['BIGDATA', 'DATA', 'processed', 'clean_llamadas_123_septiembre_2019.csv']
Total Dataframes: 6


###Generate unique table

In [ ]:
df = pd.concat(list_data)
df.head()


,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL,FECHA_INCIDENTE
0,809224194,2019-01-08 00:28:00,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado,NaN
1,809240194,2019-01-08 00:15:00,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado,NaN
2,809244194,2019-01-08 00:10:00,13,Teusaquillo,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente de tránsito con heridos/Muertos,ALTA,AGOSTO,Cancelado,NaN
3,809253194,2019-01-08 00:12:00,8,Kennedy,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente de tránsito con heridos/Muertos,ALTA,AGOSTO,Cancelado,NaN
4,809261194,2019-01-08 00:13:00,11,Suba,71,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,AGOSTO,Traslado,NaN


In [ ]:
#lista_meses = ['JULIO','DICIEMBRE','SEPTIEMBRE']
df = df[df['MES'] != 'ENERO']
df.head()

df.info()

#df[df['MES'].isin(lista_meses) == False]
#df[df['MES'].isin(lista_meses)]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95523 entries, 0 to 16266
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  95523 non-null  object
 1   CODIGO_LOCALIDAD                   95523 non-null  int64 
 2   LOCALIDAD                          95523 non-null  object
 3   EDAD                               95523 non-null  object
 4   UNIDAD                             95523 non-null  object
 5   GENERO                             88459 non-null  object
 6   RED                                95523 non-null  object
 7   TIPO_INCIDENTE                     95523 non-null  object
 8   PRIORIDAD                          95523 non-null  object
 9   MES                                30649 non-null  object
 10  CLASIFICACION_FINAL                30649 non-null  object
 11  FECHA_INCIDENTE                    13842 non-null  object
dtypes: i

In [ ]:
cols_to_drop = ['NUMERO_INCIDENTE']  # eliminar 
for col in cols_to_drop:
    try:
        df.drop(columns = col)
    except:
        print('columna ---{}--- no encontrada'.format(col))
        print(f'columna ---{col}--- no encontrada')
        continue
        
#df = df.drop(columns = cols_to_drop) eliminar columnas del dataframe
df.info()


columna ---NUMERO_INCIDENTE--- no encontrada
columna ---NUMERO_INCIDENTE--- no encontrada
<class 'pandas.core.frame.DataFrame'>
Int64Index: 95523 entries, 0 to 16266
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  95523 non-null  object
 1   CODIGO_LOCALIDAD                   95523 non-null  int64 
 2   LOCALIDAD                          95523 non-null  object
 3   EDAD                               95523 non-null  object
 4   UNIDAD                             95523 non-null  object
 5   GENERO                             88459 non-null  object
 6   RED                                95523 non-null  object
 7   TIPO_INCIDENTE                     95523 non-null  object
 8   PRIORIDAD                          95523 non-null  object
 9   MES                                30649 non-null  object
 10  CLASIFICACION_FINAL                30649

### Data imputation -- NULLS to SIN-DATO

In [ ]:
for col in df.columns:
    print(col, df[col].isna().sum())

FECHA_INICIO_DESPLAZAMIENTO_MOVIL 0
CODIGO_LOCALIDAD 0
LOCALIDAD 0
EDAD 0
UNIDAD 0
GENERO 7064
RED 0
TIPO_INCIDENTE 0
PRIORIDAD 0
MES 64874
CLASIFICACION_FINAL 64874
FECHA_INCIDENTE 81681


In [ ]:
col = 'UNIDAD'
df[col].value_counts(dropna=False)

Años        52592
SIN-DATO    42552
Meses         270
Horas          55
Dias           54
Name: UNIDAD, dtype: int64

In [ ]:
df[col].fillna('SIN-DATO', inplace=True)
df[col].replace('SIN_DATO','SIN-DATO', inplace=True)
df[col].replace('SIN/DATO','SIN-DATO', inplace=True)
df[col].value_counts(dropna=False)

Años        52592
SIN-DATO    42552
Meses         270
Horas          55
Dias           54
Name: UNIDAD, dtype: int64

###Dealing with datetimes

In [ ]:
df.head()

,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL,FECHA_INCIDENTE
0,2019-01-08 00:28:00,1,Usaquen,SIN_DATO,SIN-DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado,NaN
1,2019-01-08 00:15:00,11,Suba,SIN_DATO,SIN-DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado,NaN
2,2019-01-08 00:10:00,13,Teusaquillo,SIN_DATO,SIN-DATO,SIN_DATO,Norte,Accidente de tránsito con heridos/Muertos,ALTA,AGOSTO,Cancelado,NaN
3,2019-01-08 00:12:00,8,Kennedy,SIN_DATO,SIN-DATO,SIN_DATO,Norte,Accidente de tránsito con heridos/Muertos,ALTA,AGOSTO,Cancelado,NaN
4,2019-01-08 00:13:00,11,Suba,71,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,AGOSTO,Traslado,NaN


In [ ]:
parse(df['FECHA_INICIO_DESPLAZAMIENTO_MOVIL'].values[100])
print(parse(df['FECHA_INICIO_DESPLAZAMIENTO_MOVIL'].values[100]).time())

07:22:00


In [ ]:
ti = df[~df['FECHA_INCIDENTE'].isna()]['FECHA_INCIDENTE'].apply(parse)
tf = df[~df['FECHA_INCIDENTE'].isna()]['FECHA_INICIO_DESPLAZAMIENTO_MOVIL'].apply(parse)

deltat = tf - ti

print(deltat)

0       0 days 00:05:54
1       0 days 00:05:59
2       0 days 00:01:00
3       0 days 00:01:05
4       0 days 00:14:00
              ...      
13837   0 days 00:03:18
13838   0 days 00:06:39
13839   0 days 00:01:54
13840   0 days 00:02:49
13841   0 days 00:03:43
Length: 13842, dtype: timedelta64[ns]


In [ ]:
list_minutes = list()
for delta_time in deltat.values:
    seconds = int(delta_time)/1e9
    list_minutes.append(seconds/60.)

    
print('mean time of responde:   {} minutes'.format(np.mean(list_minutes)))
print('Median time of responde:   {} minutes'.format(np.median(list_minutes)))

mean time of responde:   17.683525983721044 minutes
Median time of responde:   6.25 minutes


In [ ]:
from dateutil.relativedelta import relativedelta

In [ ]:
ti[0] + relativedelta(minutes=np.median(list_minutes))

Timestamp('2019-07-01 00:11:23')

In [ ]:
df_out = clean_datetime_cols(df)
df_out

100%|██████████| 81681/81681 [01:08<00:00, 1189.83it/s]
/home/jupyter/ESEIT_BigData/src/data/data_manipulator.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_date_nan[initial_time_col] = list_new_date


,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL,FECHA_INCIDENTE
0,2019-01-08 00:28:00,1,Usaquen,SIN_DATO,SIN-DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado,2019-01-08 00:21:45
1,2019-01-08 00:15:00,11,Suba,SIN_DATO,SIN-DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado,2019-01-08 00:08:45
2,2019-01-08 00:10:00,13,Teusaquillo,SIN_DATO,SIN-DATO,SIN_DATO,Norte,Accidente de tránsito con heridos/Muertos,ALTA,AGOSTO,Cancelado,2019-01-08 00:03:45
3,2019-01-08 00:12:00,8,Kennedy,SIN_DATO,SIN-DATO,SIN_DATO,Norte,Accidente de tránsito con heridos/Muertos,ALTA,AGOSTO,Cancelado,2019-01-08 00:05:45
4,2019-01-08 00:13:00,11,Suba,71,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,AGOSTO,Traslado,2019-01-08 00:06:45
...,...,...,...,...,...,...,...,...,...,...,...,...
13837,2019-07-31 23:21:39,10,Engativa,SIN_DATO,SIN-DATO,SIN_DATO,Norte,Heridos,ALTA,JULIO,Cancelado,2019-07-31 23:18:21
13838,2019-07-31 23:28:20,18,Rafael Uribe Uribe,52,Años,FEMENINO,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA,JULIO,Traslado,2019-07-31 23:21:41
13839,2019-07-31 23:27:43,13,Teusaquillo,SIN_DATO,SIN-DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,JULIO,Cancelado,2019-07-31 23:25:49
13840,2019-07-31 23:29:43,18,Rafael Uribe Uribe,78,Años,MASCULINO,Sur,Convulsiones,ALTA,JULIO,Desistimiento,2019-07-31 23:26:54


In [ ]:
def standarize_datetimes(x):
    try:
        new_x = parse(x)
    except:
        new_x = x
    return new_x

In [ ]:
df_out['FECHA_INICIO_DESPLAZAMIENTO_MOVIL'] = df.out['FECHA_INICIO_DESPLAZAMIENTO_MOVIL']

In [ ]:
import unidecode

unicode.unicode

SyntaxError: invalid syntax (2405712113.py, line 3)